In [1]:
# feel free to comment out this cell

import sys
assert sys.version.split(" ")[0] == "3.9.6"

In [13]:
import random

class Door:
    
    def __init__(self, door_num, is_prize=False):
        # private
        self.__door_num = door_num
        self.__is_prize = is_prize
        # protected
        self._is_revealed = False
    
    def get_door_num(self):
        return self.__door_num
    
    def is_revealed(self):
        return self._is_revealed
    
    def is_prize(self):
        if self._is_revealed:
            return self.__is_prize
        raise Exception("tried to check prize on unrevealed door")
    

class AbstractPlayer:
    
    def __init__(self):
        self._first_choice = None
    
    def start_new_game(self):
        self._first_choice = None
    
    def end_game(self, result):
        pass
        
    def choose_door(doors):
        raise Exception("not implemented")

class HumanPlayer(AbstractPlayer):
    
    def start_new_game(self):
        print("Starting new game!")
    
    def end_game(self, result):
        print("Ending game.")
        print(f"Result: {result}")
        
    def choose_door(self, doors: list[Door]) -> int:
        
        door_labels = []
        for door in doors:
            door_label = str(door.get_door_num()) + ": "
            if not door.is_revealed():
                door_label += "(hidden)"
            elif not door.is_prize():
                door_label += "goat"
            else:
                door_label += "prize"
                
            door_labels.append(door_label)
            
        print(f"Door choices: {', '.join(door_labels)}")
        
        while True:
            
            choice = input("Which door do you choose?")
            try:
                choice = int(choice)
            except Exception as e:
                print(f"Encountered exception: {e}")
                continue
                
            if choice not in [door.get_door_num() for door in doors]:
                print("Invalid choice.")
                continue
            
            if self._first_choice is not None:
                self._first_choice = choice
                
            return choice

class RobotSwitcher(AbstractPlayer):
    
    def choose_door(self, doors: list[Door]) -> int:
        choices = []
        for door in doors:
            if door.is_revealed() and door.is_prize():
                print("potential problem: prize door is revealed")
                return door.get_door_num()
            if door.is_revealed() and not door.is_prize():
                continue
            if door.get_door_num() != self._first_choice:
                choices.append(door.get_door_num())
        
        choice = random.choice(choices)
        
        if self._first_choice is None:
            self._first_choice = choice
        
        return choice

class RobotNonSwitcher(AbstractPlayer):
    
    def choose_door(self, doors: list[Door]) -> int:
        if self._first_choice is None:
            choices = []
            for door in doors:
                if door.is_revealed() and door.is_prize():
                    print("potential problem: prize door is revealed")
                    return door.get_door_num()
                return door.get_door_num()
                if door.is_revealed() and not door.is_prize():
                    continue
                choices.append(door.get_door_num())
            choice = random.choice(choices)
            self._first_choice = choice
        return self._first_choice

class RobotRandomChooser(AbstractPlayer):
    
    def choose_door(self, doors: list[Door]) -> int:
        choices = []
        for door in doors:
            if door.is_revealed() and door.is_prize():
                print("potential problem: prize door is revealed")
                return door.get_door_num()
            if door.is_revealed() and not door.is_prize():
                continue
            choices.append(door.get_door_num())
        
        choice = random.choice(choices)
        
        if self._first_choice is None:
            self._first_choice = choice
        
        return choice
        
class Game:
    
    def __init__(self, player: AbstractPlayer, num_doors=3, num_reveal=1):
        
        assert isinstance(num_doors, int) and num_doors >= 3, num_doors
        assert isinstance(num_reveal, int) and num_reveal >= 1 and num_reveal < num_doors, num_reveal
        
        self.__player = player
        
        self.__prize_num = random.randint(1, num_doors)
        self.__doors = [Door(ix, is_prize=(ix == self.__prize_num)) for ix in range(1, num_doors+1)]
        self.__door_num_to_door = {door.get_door_num(): door for door in self.__doors}
        
        self.__num_reveal = num_reveal
        self.__player_first_choice = None
        self.__player_final_choice = None
        
        self.__is_game_started = False
        self.__is_game_finished = False
    
    def play_game(self):
        self._start_game()
        self._offer_first_choice()
        self._reveal_doors()
        self._offer_final_choice()
        self._finish_game()
        return self.did_player_win()
    
    def did_player_change_choice(self):
        assert self.__player_first_choice is not None
        assert self.__player_final_choice is not None
        assert self.__is_game_finished
        return self.__player_first_choice != self.__player_final_choice
    
    def did_player_win(self):
        assert self.__player_final_choice is not None
        assert self.__is_game_finished
        return self.__player_final_choice == self.__prize_num
    
    def _start_game(self):
        assert not self.__is_game_started and not self.__is_game_finished
        self.__is_game_started = True
        self.__player.start_new_game()
    
    def _finish_game(self):
        assert self.__is_game_started and not self.__is_game_finished
        self.__is_game_finished = True
        self.__player.end_game(self.did_player_win())
        
    def _offer_first_choice(self):
        assert self.__is_game_started and not self.__is_game_finished
        self.__player_first_choice = self.__player.choose_door(self.__doors)
    
    def _reveal_doors(self):
        assert self.__is_game_started and not self.__is_game_finished
        possible_doors_to_reveal = list(self.__door_num_to_door.keys())
        possible_doors_to_reveal.remove(self.__prize_num)
        if self.__player_first_choice in possible_doors_to_reveal:
            possible_doors_to_reveal.remove(self.__player_first_choice)
        for _ in range(self.__num_reveal):
            door_to_reveal = random.choice(possible_doors_to_reveal)
            possible_doors_to_reveal.remove(door_to_reveal)
            setattr(self.__door_num_to_door[door_to_reveal], "_is_revealed", True)
    
    def _offer_final_choice(self):
        assert self.__is_game_started and not self.__is_game_finished
        self.__player_final_choice = self.__player.choose_door(self.__doors)

def analyzeGames(games: list[Game]):
    num_games = float(len(games))
    num_won = float(sum(game.did_player_win() for game in games))
    num_switched = float(sum(game.did_player_change_choice() for game in games))
    num_not_switched = float(sum(not game.did_player_change_choice() for game in games))
    num_won_out_of_switched = float(sum(game.did_player_win() for game in games if game.did_player_change_choice()))
    num_won_out_of_not_switched = float(sum(game.did_player_win() for game in games if not game.did_player_change_choice()))
    num_switched_out_of_won = float(sum(game.did_player_change_choice() for game in games if game.did_player_win()))
    num_not_switched_out_of_won = float(sum(not game.did_player_change_choice() for game in games if game.did_player_win()))
    print(f"Player won {num_won / num_games} of games")
    print(f"Player switched choice in {num_switched / num_games} of games")
    if num_switched > 0:
        print(f"Player won {num_won_out_of_switched / num_switched} of games in which they switched choice")
    if num_not_switched > 0:
        print(f"Player won {num_won_out_of_not_switched / num_not_switched} of games in which they did not switch choice")
    if num_won > 0:
        print(f"Player switched choice in {num_switched_out_of_won / num_won} of games in which they won")
    if num_won > 0:
        print(f"Player did not switch choice in {num_not_switched_out_of_won / num_won} of games in which they won")

In [14]:
NUM_GAMES = 1000

In [15]:
player = RobotSwitcher()

games = [Game(player) for _ in range(NUM_GAMES)]
for game in games:
    game.play_game()

analyzeGames(games)

Player won 0.638 of games
Player switched choice in 1.0 of games
Player won 0.638 of games in which they switched choice
Player switched choice in 1.0 of games in which they won
Player did not switch choice in 0.0 of games in which they won


In [16]:
player = RobotNonSwitcher()

games = [Game(player) for _ in range(NUM_GAMES)]
for game in games:
    game.play_game()

analyzeGames(games)

Player won 0.347 of games
Player switched choice in 0.0 of games
Player won 0.347 of games in which they did not switch choice
Player switched choice in 0.0 of games in which they won
Player did not switch choice in 1.0 of games in which they won


In [17]:
player = RobotRandomChooser()

games = [Game(player) for _ in range(NUM_GAMES)]
for game in games:
    game.play_game()

analyzeGames(games)

Player won 0.494 of games
Player switched choice in 0.516 of games
Player won 0.6569767441860465 of games in which they switched choice
Player won 0.3202479338842975 of games in which they did not switch choice
Player switched choice in 0.6862348178137652 of games in which they won
Player did not switch choice in 0.31376518218623484 of games in which they won
